In [12]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit.circuit.library import ZFeatureMap, EfficientSU2
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_ibm_provider import least_busy
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import SLSQP
from qiskit_ibm_runtime import Sampler
from qiskit_ibm_runtime import Options
from qiskit_ibm_runtime import QiskitRuntimeService

# Create sampler object
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q-asu/main/pi-deluca',
)
backend = service.backend('simulator_statevector')
options = Options()
options.execution.shots = 1024
sampler = Sampler(session=backend, options=options)

# Load the Iris dataset and preprocess the data
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Get number of qubits from number of features
num_qubits = X.shape[1]

# Define the feature map and ansatz
feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=2)
ansatz = EfficientSU2(num_qubits=num_qubits, reps=1)

# Quantum circuit
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

def interpret_measurement(x):
    return x % 3

# Set up the sampler qnn
qnn = SamplerQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=interpret_measurement,
    output_shape=3,
    sampler=sampler
)

# Set up the neural network classifier
classifier = NeuralNetworkClassifier(qnn, loss='cross_entropy', one_hot=True, optimizer=SLSQP(maxiter=50, max_evals_grouped=3), warm_start=True)

# Train the classifier
classifier.fit(X_train, y_train)

# Test the classifier
accuracy = classifier.score(X_test, y_test)
print("Test accuracy:", accuracy)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Test accuracy: 0.8
